In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.9 MB/s eta 0:00:00


# Service 1

In [3]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler

# Load dataset
df = pd.read_csv("Service1.csv")

# Input and output
X = df[['latency_ms', 'cpu_usage_pct', 'memory_usage_pct']]
y = df[['cpu_allocated', 'memory_allocated']]

# Split: 70% train, 15% val, 15% test
X_train_full, X_temp, y_train_full, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale features
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = x_scaler.fit_transform(X_train_full)
X_val_scaled = x_scaler.transform(X_val)
X_test_scaled = x_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train_full)
y_val_scaled = y_scaler.transform(y_val)

# Objective for Optuna
def objective(trial):
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    epsilon = trial.suggest_float('epsilon', 0.001, 1.0, log=True)
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    svr_model = MultiOutputRegressor(SVR(C=C, epsilon=epsilon, gamma=gamma))
    svr_model.fit(X_train_scaled, y_train_scaled)
    preds_scaled = svr_model.predict(X_val_scaled)
    mse = mean_squared_error(y_val_scaled, preds_scaled)
    return mse

# Run optimization
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=30)

# Best model training
best_params = study.best_params
print("Best Parameters:", best_params)

final_svr = MultiOutputRegressor(SVR(**best_params))
final_svr.fit(np.vstack([X_train_scaled, X_val_scaled]), np.vstack([y_train_scaled, y_val_scaled]))

# Test set evaluation
y_test_scaled = y_scaler.transform(y_test)
preds_scaled = final_svr.predict(X_test_scaled)
preds = y_scaler.inverse_transform(preds_scaled)

mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
print(f"\nTest MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")

# Sample prediction
sample = pd.DataFrame([[300, 40, 75]], columns=['latency_ms', 'cpu_usage_pct', 'memory_usage_pct'])
sample_scaled = x_scaler.transform(sample)
predicted_scaled = final_svr.predict(sample_scaled)
predicted = y_scaler.inverse_transform(predicted_scaled)

print(f"\nSample Input: {sample.to_dict(orient='records')[0]}")
print(f"Predicted CPU, Memory Allocation: CPU={predicted[0][0]:.3f}, Memory={predicted[0][1]:.2f}")


[I 2025-05-05 05:47:14,088] A new study created in memory with name: no-name-b03b1599-827d-4191-88ef-6338c1f6329e
[I 2025-05-05 05:47:26,877] Trial 0 finished with value: 0.010229546466552645 and parameters: {'C': 1.050170101231293, 'epsilon': 0.02685913346943676, 'gamma': 'scale'}. Best is trial 0 with value: 0.010229546466552645.
[I 2025-05-05 05:47:36,121] Trial 1 finished with value: 0.01663855296687673 and parameters: {'C': 0.12985768446507892, 'epsilon': 0.004731901110444143, 'gamma': 'scale'}. Best is trial 0 with value: 0.010229546466552645.
[I 2025-05-05 05:47:36,147] Trial 2 finished with value: 0.2537107519083336 and parameters: {'C': 0.18194925221233044, 'epsilon': 0.817641580774545, 'gamma': 'auto'}. Best is trial 0 with value: 0.010229546466552645.
[I 2025-05-05 05:47:36,182] Trial 3 finished with value: 0.0647681212714494 and parameters: {'C': 3.1646360702756478, 'epsilon': 0.39910802837195825, 'gamma': 'auto'}. Best is trial 0 with value: 0.010229546466552645.
[I 2025-0

Best Parameters: {'C': 92.3299622855522, 'epsilon': 0.018072910114645152, 'gamma': 'auto'}

Test MSE: 679.2882
Test RMSE: 26.0632

Sample Input: {'latency_ms': 300, 'cpu_usage_pct': 40, 'memory_usage_pct': 75}
Predicted CPU, Memory Allocation: CPU=0.784, Memory=1771.37
